# Mojito noise pipeline with the inclusion of gaps

Very simple L01 pipeline, containing
* reading of the data (telemetry, ground tracking, and instrument knowledge)
* optionally correction for group delays
* directly use the PRN measurements as MPRs, and compute a 2-point difference
  for the MPR derivatives
* calculation of intermediary and Michelson TDI variables with gaps treated as nans.

This pipeline uses data saved in adjacent HDF5 files, saved on Git LFS. To
re-generate the data, use the adjacent `simulation.ipynb` notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import yaml
import logging
import numpy as np
import matplotlib.pyplot as plt
import h5py

from lolipops.conventions import MOSAS
from lolipops.pipelines import welch, tdi_secondaries
from lolipops.constellation.tdi import (
    compute_factorized_intervars,
    compute_factorized_michelson,
    trim_tdis,
)
from lolipops.local import ifo, prn

from lolipops.io import read_telemetry_from_instrument, read_ground_tracking, Sampling

logging.basicConfig(level=logging.INFO)
logging.getLogger("pytdi").setLevel(logging.WARNING)

from LISA_artefacts import GapMaskGenerator

## Read data

In [ ]:
telemetry_file = "data/simple/instru.h5"
ground_tracking_file = "data/simple/groundtrack.h5"

telemetry, instrument_knowledge = read_telemetry_from_instrument(telemetry_file)
ground_tracking = read_ground_tracking(ground_tracking_file, telemetry_file)

orbits_file = "data/simple/orbits.h5"

In [ ]:
# Configuration file
config_file = "data/simple/config.yaml"

with Path(config_file).open("r") as file:
    config_dict = yaml.safe_load(file)

## IFO control plots

In [ ]:
# Telemetry data and apply group delay corrections
ifos = telemetry.ifos
prns = telemetry.prns

# Group delay correction
corrected_ifos = ifo.cancel_filter_group_delay(
    ifos, telemetry.sampling, telemetry.instrument_knowledge
)
corrected_prns = prn.cancel_filter_group_delay(
    prns, telemetry.sampling, telemetry.instrument_knowledge
)

# Time
t = telemetry.sampling.t()

In [ ]:
# IFO measurements control plots - time domain

STOP = -1
STEP = 100

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].plot(t[:STOP:STEP], ifos[f"isi_{mosa}"][:STOP:STEP], label=mosa)
    axs[1].plot(t[:STOP:STEP], corrected_ifos[f"isi_{mosa}"][:STOP:STEP], label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("Beatnote [Hz]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Time [s]")
axs[1].set_title("After group delay correction")
fig.suptitle("ISI Carrier Beatnotes")
plt.show()

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].plot(t[:STOP:STEP], ifos[f"tmi_{mosa}"][:STOP:STEP], label=mosa)
    axs[1].plot(t[:STOP:STEP], corrected_ifos[f"tmi_{mosa}"][:STOP:STEP], label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("Beatnote [Hz]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Time [s]")
axs[1].set_title("After group delay correction")
fig.suptitle("TMI Carrier Beatnotes")
plt.show()

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].plot(t[:STOP:STEP], ifos[f"rfi_{mosa}"][:STOP:STEP], label=mosa)
    axs[1].plot(t[:STOP:STEP], corrected_ifos[f"rfi_{mosa}"][:STOP:STEP], label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("Beatnote [Hz]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Time [s]")
axs[1].set_title("After group delay correction")
fig.suptitle("RFI Carrier Beatnotes")
plt.show()

In [ ]:
# IFO measurements control plots - spectral estimate

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].loglog(*welch(ifos[f"isi_{mosa}"], telemetry.sampling), label=mosa)
    axs[1].loglog(*welch(corrected_ifos[f"isi_{mosa}"], telemetry.sampling), label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Frequency [Hz]")
axs[0].set_ylabel("ASD [Hz/sqrt(Hz)]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_title("After group delay correction")
fig.suptitle("ISI Carrier Beatnotes")
plt.show()

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].loglog(*welch(ifos[f"tmi_{mosa}"], telemetry.sampling), label=mosa)
    axs[1].loglog(*welch(corrected_ifos[f"tmi_{mosa}"], telemetry.sampling), label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Frequency [Hz]")
axs[0].set_ylabel("ASD [Hz/sqrt(Hz)]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_title("After group delay correction")
fig.suptitle("TMI Carrier Beatnotes")
plt.show()

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].loglog(*welch(ifos[f"rfi_{mosa}"], telemetry.sampling), label=mosa)
    axs[1].loglog(*welch(corrected_ifos[f"rfi_{mosa}"], telemetry.sampling), label=mosa)
axs[0].legend(loc="upper right")
axs[0].set_xlabel("Frequency [Hz]")
axs[0].set_ylabel("ASD [Hz/sqrt(Hz)]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="upper right")
axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_title("After group delay correction")
fig.suptitle("RFI Carrier Beatnotes")
plt.show()

In [ ]:
# PRN control plot

fig, axs = plt.subplots(
    1, 2, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    axs[0].plot(t[:STOP:STEP], prns[mosa][:STOP:STEP], label=mosa)
    axs[1].plot(t[:STOP:STEP], corrected_prns[mosa][:STOP:STEP], label=mosa)
axs[0].legend(loc="center right")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("PRN [s]")
axs[0].set_title("Before group delay correction")
axs[1].legend(loc="center right")
axs[1].set_xlabel("Time [s]")
axs[1].set_title("After group delay correction")
fig.suptitle("PRN Carrier Beatnotes")
plt.show()

## MPR control plots

In [ ]:
mprs = corrected_prns

mpr_derivatives = {
    mosa: np.gradient(mprs[mosa], telemetry.sampling.dt) for mosa in MOSAS
}

In [ ]:
with h5py.File(orbits_file, "r") as f:
    true_t = f.attrs["t0"] + np.arange(f.attrs["size"]) * f.attrs["dt"]
    true_ltts = f["tcb/ltt"][:]
    true_ltt_derivatives = f["tcb/d_ltt"][:]

# Restrict true_t to the telemetry time range
start_index = np.where(true_t >= t[0])[0][0] - 1
end_index = np.where(true_t <= t[-1])[0][-1] + 2
true_t = true_t[start_index:end_index]
true_ltts = true_ltts[start_index:end_index]
true_ltt_derivatives = true_ltt_derivatives[start_index:end_index]

In [ ]:
plt.figure(figsize=(8, 5))
colors = iter(plt.rcParams["axes.prop_cycle"].by_key()["color"])
for i, mosa in enumerate(MOSAS):
    color = next(colors)
    plt.plot(t[:STOP:STEP], mprs[mosa][:STOP:STEP], c=color, label=f"{mosa} (estim.)")
    plt.plot(true_t, true_ltts[:, i], "o", c=color, label=f"{mosa} (true)")
plt.legend(loc="upper right")
plt.xlabel("Time [s]")
plt.ylabel("MPR [s]")
plt.title("MPR Estimates")
plt.show()

plt.figure(figsize=(8, 5))
colors = iter(plt.rcParams["axes.prop_cycle"].by_key()["color"])
for i, mosa in enumerate(MOSAS):
    color = next(colors)
    plt.plot(
        t[:STOP:STEP],
        mpr_derivatives[mosa][:STOP:STEP],
        c=color,
        label=f"{mosa} (estim.)",
    )
    plt.plot(true_t, true_ltt_derivatives[:, i], "o", c=color, label=f"{mosa} (true)")
plt.legend(loc="upper right")
plt.xlabel("Time [s]")
plt.ylabel("MPR Derivative [s/s]")
plt.title("MPR Derivative Estimates")
plt.show()

# Build the gap function! 

In [ ]:
# Construct gap class

SPECIFIC_NANS = False
# from gap_mask_generation import GapMaskGenerator

N = telemetry.sampling.size
t_start = telemetry.sampling.t0

sim_t = np.arange(0, N * telemetry.sampling.dt, telemetry.sampling.dt)

gap_definitions = {
    "planned": {"Aliens": {"rate_per_year": 200, "duration_hr": 12}},
    "unplanned": {"None": {"rate_per_year": 0, "duration_hr": 0}},
}


# Initialise class with rates, and sampling properties.
generator = GapMaskGenerator(
    sim_t,
    telemetry.sampling.dt,
    gap_definitions,
    treat_as_nan=True,
    planseed=11_07_1993,
)

# Generate the mask full of gaps
full_mask = generator.generate_mask(include_unplanned=False, include_planned=True)

if SPECIFIC_NANS:
    mask_mult_nan = np.ones(len(full_mask))
    mid_index = int(len(mask_mult_nan) / 2)
    mask_mult_nan[mid_index : mid_index + 1] = np.nan
    masking_function = mask_mult_nan
else:
    masking_function = full_mask

plt.clf()

plt.plot(sim_t / 60 / 60 / 24, masking_function)
plt.title("Full gap mask")
plt.xlabel("Time [day]")
plt.ylabel("Gap mask")
plt.grid(True)

print(
    "Lost out on approximately",
    np.sum(np.isnan(masking_function)) * 0.25 / 60 / 60,
    "hours of data",
)
print(
    "This ammounts to {} number of samples".format(np.sum(np.isnan(masking_function)))
)

In [ ]:
# Place the gaps in the telemetry variables
import copy

# Make a copy of the telemetric data variables
telemetry_w_gaps = copy.deepcopy(telemetry)

idx_start_gap = np.argwhere(np.isnan(masking_function) == True)[0][0]
idx_end_gap = np.argwhere(np.isnan(masking_function) == True)[-1][-1]


print("Starting and ending indices of the gap in the telemetry data:")
print(idx_start_gap, idx_end_gap)

tmi_label = [f"tmi_{mosa}" for mosa in MOSAS]
rfi_label = [f"rfi_{mosa}" for mosa in MOSAS]
rfi_usb_label = [f"rfi_usb_{mosa}" for mosa in MOSAS]
isi_label = [f"isi_{mosa}" for mosa in MOSAS]
isi_usb_label = [f"isi_usb_{mosa}" for mosa in MOSAS]

for tmi_item, rfi_item, rfi_usb_item, isi_item, isi_usb_item in zip(
    tmi_label, rfi_label, rfi_usb_label, isi_label, isi_usb_label
):
    telemetry_w_gaps.ifos[tmi_item] = (
        telemetry.ifos[tmi_item] * masking_function
    )  # No delays in each of these.
    telemetry_w_gaps.ifos[rfi_item] = (
        telemetry.ifos[rfi_item] * masking_function
    )  # No delays in each of these.
    telemetry_w_gaps.ifos[rfi_usb_item] = (
        telemetry.ifos[rfi_usb_item] * masking_function
    )  # No delays in each of these.
    telemetry_w_gaps.ifos[isi_item] = (
        telemetry.ifos[isi_item] * masking_function
    )  # No delays in each of these.
    telemetry_w_gaps.ifos[isi_usb_item] = (
        telemetry.ifos[isi_usb_item] * masking_function
    )  # No delays in each of these.

plt.clf()

# Plot
plt.plot(sim_t, telemetry.ifos["tmi_23"], label="no gaps")
plt.plot(sim_t, telemetry_w_gaps.ifos["tmi_23"], "*", label="with gaps", alpha=0.1)
plt.ylabel(r"TMI_23")
plt.xlabel("Time [s]")
plt.legend()

## TDIs, first compute intermediary variables without gaps, and with gaps treated as nans and zeros!

In [ ]:
# Without gaps in the intermediary variables
etas = compute_factorized_intervars(
    telemetry.ifos, mprs, mpr_derivatives, telemetry.sampling, lagrange_interp_order=45
)

In [ ]:
# With gaps in the intermediary variables
lagrange_order_eta_gap = 81
etas_w_gaps_nans = compute_factorized_intervars(
    telemetry_w_gaps.ifos,
    mprs,
    mpr_derivatives,
    telemetry.sampling,
    lagrange_interp_order=lagrange_order_eta_gap,
)

In [ ]:
for mosa in MOSAS:
    N_gap_extension_eta = np.sum(
        np.isnan(etas_w_gaps_nans["eta_" + str(mosa)])
    ) - np.sum(np.isnan(masking_function))
    print(f"For eta_{mosa}, the gap extension is {N_gap_extension_eta} extra samples")

print("")
print(f"The order of the lagrange interpolant is {lagrange_order_eta_gap}")
print(
    f"Total number of nans in etas with gaps: {np.sum(np.isnan(etas_w_gaps_nans["eta_12"]))}"
)
print(
    f"Total number of nans in telemetry variables with gaps: {np.sum(np.isnan(telemetry_w_gaps.ifos['tmi_12']))}"
)

Notice that the gap augmentation (widening) is exactly equal to the order of the Lagrange interpolant in this case. The results will change depending on 

* The values of the measured pseudo-ranges (MPRs), i.e., the light travel times on the arm-lengths separating the MOSAs
* The order of the interpolant itself
* The number of consecutive missing data
* the number of gaps. 

Our formula to predict the gap augmentation goes like

\begin{equation}
N^{\text{tel} \rightarrow \eta}_{\text{lost samples}} = \begin{cases}
    \mathcal{O}(L)_{\eta} + N^{\text{tel}}_{\texttt{nans}} & \text{if} \  1 < \mathcal{O}(L)_{\eta} \leq 65 - 2N^{\text{tel}}_{\texttt{nans}} \\
    \frac{\mathcal{O}(L)_{\eta} + 1}{2} + \lfloor \frac{L}{c\Delta t}\rfloor  & 67 - 2N^{\text{tel}}_{\texttt{nans}} \leq \mathcal{O}(L)_{\eta} \leq 67 \\
    \mathcal{O}(L)_{\eta} &  \mathcal{O}(L)_{\eta} \geq 67\,,
\end{cases}
\end{equation}

where $L$ is the arm-length, $c$ the speed of light and $\Delta t$ the sampling interval. Here $67 = \lfloor 2L/(c\Delta t)\rfloor$ and $N^{\text{tel}}_{\texttt{nans}}$ is the number of nans in the original telemetry variables. Predicting the total number of missing data in the telemetry variables is easy, you just need to add $N^{\text{tel}}_{\texttt{nans}}$ to the piecewise expression above. 

In [ ]:
N_nans_in_telemetry_variables = np.sum(np.isnan(masking_function))
print("Number of nans in telemetry function = ", N_nans_in_telemetry_variables)
print("Order of lagrange interpolant", lagrange_order_eta_gap)
print("Number of nans in eta variables", np.sum(np.isnan(etas_w_gaps_nans["eta_12"])))

## Plot the data sets

Notice if you treat the eta variables as zeros and applying a welch estimate is horribly plagued by artefacts. I would not look at the spectrum of the eta variables unless one applies careful tapers etc. Though, I believe this is unecessary! The nan approach seems reasonable.  

In [ ]:
plt.figure(figsize=(8, 5))

fig, ax = plt.subplots(
    1, 1, figsize=(10, 5), sharex=True, sharey=True, tight_layout=True
)
for mosa in MOSAS:
    ax.loglog(*welch(etas[f"eta_{mosa}"], telemetry.sampling), label=mosa)

for j in range(2):
    ax.legend()
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("ASD [Hz/sqrt(Hz)]")
    ax.set_title("ETA Intervars, no gaps")
    ax.set_title("ETA Intervars, gaps w/ zeros")

plt.show()
print(MOSAS)

In [ ]:
freq = np.logspace(-5, 0, 1000)
alloc = tdi_secondaries(freq, model="SciRD")

noise = tdi_secondaries(
    freq,
    model=None,
    oms_level=config_dict["instru"]["oms_asds"][0],
    tm_level=config_dict["instru"]["testmass_asds"],
)

# Time-Delay Interferometry with gaps

Now we can take our intermediary variables ($\eta$) with missing data and propagate them through the factorized TDI expressions. We first compute the TDI expressions without missing data and then missing data. 

In [ ]:
tdis = compute_factorized_michelson(
    etas,
    mprs,
    mpr_derivatives,
    telemetry.sampling,
    generation=2,
    lagrange_interp_order=45,
)
trimmed_tdis, tdi_sampling = trim_tdis(tdis, telemetry.sampling, samples=1000)

In [ ]:
# Incorporate gaps into the Michelson like TDI variables using NaNs

lagrange_order_michelson_gap = 81
tdis_w_gaps = compute_factorized_michelson(
    etas_w_gaps_nans,
    mprs,
    mpr_derivatives,
    telemetry.sampling,
    generation=2,
    lagrange_interp_order=lagrange_order_michelson_gap,
)
trimmed_tdis_w_gaps_nans, tdi_sampling = trim_tdis(
    tdis_w_gaps, telemetry.sampling, samples=1000
)

# What if we treat missing data as zeros?

In the following, we will treat the missing data in the $\eta$ variables as zeros. We will show this leads to nasty sharp transitions in the output TDI variables

In [ ]:
# Incorporate gaps into the Michelson like TDI variables using zeros
etas_w_gaps_zeros = {k: np.nan_to_num(v, nan=0.0) for k, v in etas_w_gaps_nans.items()}
tdis_w_gaps_zeros = compute_factorized_michelson(
    etas_w_gaps_zeros,
    mprs,
    mpr_derivatives,
    telemetry.sampling,
    generation=2,
    lagrange_interp_order=101,
)
trimmed_tdis_w_gaps_zeros, tdi_sampling = trim_tdis(
    tdis_w_gaps_zeros, telemetry.sampling, samples=1000
)

In [ ]:
# Plot the trimmed TDI X variable

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, tight_layout=True)
for j in range(2):
    ax[j].plot(
        tdi_sampling.t() / 60 / 60 / 24,
        trimmed_tdis["X"][:],
        label="no gaps",
    )
    ax[j].set_ylabel(r"TDI X")
    ax[j].set_xlabel("Time [day]")
    ax[j].grid(True)

# Plot TDI X variables with gaps and with nans
ax[0].plot(
    tdi_sampling.t() / 60 / 60 / 24,
    trimmed_tdis_w_gaps_nans["X"][:],
    label="with gaps",
)
ax[1].plot(
    tdi_sampling.t() / 60 / 60 / 24,
    trimmed_tdis_w_gaps_zeros["X"][:],
    label="with gaps",
)
for j in range(2):
    ax[j].legend()

ax[0].set_title("TDI X variable w/ nans")
ax[1].set_title("TDI X variable w/ zeros")
plt.show()

In [ ]:
# Zoom in on the TDI X variable when zeros have been propagated through
plt.clf()
fig, ax = plt.subplots(2, 2, figsize=(10, 5), tight_layout=True)
for j in range(2):
    for i in range(2):
        ax[i][j].plot(
            trimmed_tdis["X"][:],
            label="no gaps",
        )
        ax[i][j].plot(
            trimmed_tdis_w_gaps_zeros["X"][:],
            label="with gaps",
        )
        ax[i][j].plot(
            trimmed_tdis_w_gaps_nans["X"][:],
            label="with nans",
        )

        ax[i][j].set_ylabel(r"TDI X")
        ax[i][j].set_xlabel("Time [day]")
        ax[i][j].grid(True)
        ax[i][j].legend()

# Plot TDI X variables with gaps and with nans


ax[0][0].set_xlim([6.355e5, 6.36e5])
ax[0][1].set_xlim([7.22e5, 7.23e5])

ax[1][0].set_xlim([6.359e5, 6.362e5])
ax[1][0].set_ylim([-0.05, 0.05])
ax[0][1].set_xlim([7.22e5, 7.23e5])
ax[1][1].set_xlim([6.358e5, 6.359e5])
ax[1][1].set_ylim([-100, 100])
plt.show()

In [ ]:
plt.figure(figsize=(11, 6))
plt.loglog(*welch(trimmed_tdis["X"], tdi_sampling), alpha=0.7, label="X2")
plt.loglog(*welch(trimmed_tdis["Y"], tdi_sampling), alpha=0.7, label="Y2")
plt.loglog(*welch(trimmed_tdis["Z"], tdi_sampling), alpha=0.7, label="Z2")
plt.loglog(freq, alloc, c="tab:red", label="Allocation (SciRD)")
plt.loglog(freq, noise, c="black", ls="--", label="Noise model")
plt.legend()
plt.xlabel("Frequency [Hz]")
plt.ylabel("ASD [Hz/sqrt(Hz)]")
plt.title("XYZ TDI Variables")
plt.show()

# Can we try estimate, roughly speaking, how much wider the gap is? 

## My conjecture:

Let $\mathcal{O}(\mathcal{L}_{\eta})$ and $\mathcal{O}(\mathcal{L}_{X})$ represent the order of the Lagrange interpolant for 
For TDI2, I suspect that we will lose out on precisely

$$ N_{\text{lost samples}} = (N_{\text{segments}} - 1)(\mathcal{O}(L)_{\eta} + 2\mathcal{O}(L)_{X} + 6 \left\lfloor\frac{L}{c\Delta t}\right\rfloor)$$ 

For TDI1, the formula I seem to have is 

$$ N_{\text{lost samples}} = (N_{\text{segments}} - 1)(\mathcal{O}(L)_{\eta} + \frac{3}{2}\mathcal{O}(L)_{X} + 2 \left\lfloor\frac{L}{c\Delta t}\right\rfloor + \frac{1}{2})$$ 

In [ ]:
# In the original masking function.

print("For the original masking function")
print(
    "Lost out on approximately",
    np.sum(np.isnan(masking_function)) * 0.25 / 60 / 60,
    "hours of data",
)
print(
    "This ammounts to {} number of samples".format(np.sum(np.isnan(masking_function)))
)

print("")
print("For the eta variables")
print(
    "Lost out on approximately",
    np.sum(np.isnan(etas_w_gaps_nans["eta_23"])) * 0.25 / 60 / 60,
    "hours of data",
)
print(
    "This ammounts to {} number of samples".format(
        np.sum(np.isnan(etas_w_gaps_nans["eta_23"]))
    )
)

print(
    f"Now, comparing the two quantities, we have lose out on {np.sum(np.isnan(etas_w_gaps_nans["eta_23"])) - np.sum(np.isnan(masking_function))} samples"
)

print(f"Notice that this should be equal to {lagrange_order_eta_gap}")
print("")
print("For the TDI variables")
print(
    "Lost out on approximately",
    np.sum(np.isnan(trimmed_tdis_w_gaps_nans["X"])) * 0.25 / 60 / 60,
    "hours of data",
)
print(
    "This ammounts to {} number of samples".format(
        np.sum(np.isnan(trimmed_tdis_w_gaps_nans["X"]))
    )
)
print(
    f"Now, comparing to the masking function, we have lose out on {np.sum(np.isnan(trimmed_tdis_w_gaps_nans["X"])) - np.sum(np.isnan(masking_function))} samples"
)

from lisaconstants import c

LTT = 2.5e9 / c

guess_TDI_2 = LTT * 6
guess_TDI_1 = LTT * 4
print(
    f"Notice that this should be equal to {(1)*(lagrange_order_eta_gap + 2*lagrange_order_michelson_gap + int(6*2.5e9/(c*telemetry.sampling.dt)) - 1)}"
)

# Return quality data flags

In particular, we can take the resultant time-series for X, Y and Z and build the true masking function that can be passed downstream to L2A/L2D etc. 

In [ ]:
# Build quality data flags

Masking_Function_X = generator.build_quality_flags(trimmed_tdis_w_gaps_nans["X"][:])
print(
    "Number of quality flags in telemetry time-series: ",
    np.sum(np.isnan(masking_function)),
)
print(
    "Number of quality flags in TDI X variable: ", np.sum(np.isnan(Masking_Function_X))
)

print(
    "Widening of the gaps from telemetry to TDIX:",
    np.sum(np.isnan(Masking_Function_X)) - np.sum(np.isnan(masking_function)),
)